# 🦠 COVID-19 in California
>A tracker for number of tests performed, coronavirus deaths and hospital cases in California 📊

- badges: true
- categories: [altair, data vis, covid19]
- show_tags: true
- image: images/layeredAreaChart.png
- toc: true


# 🧪 ELI5: testing
I thought I'd begin with a quick overview of how the tests are performed in case anyone's interested. So the way we detect COVID-19 is by running a [PCR](https://en.wikipedia.org/wiki/Polymerase_chain_reaction) (polymerase chain reaction), which is a really common technique in molecular biology. 

* You start by collecting a sample, like a nose swab, which returns boogers and things. If the virus is present, it's likely that some of it will be trapped there. 
* Since the coronavirus is a RNA virus, you need to separate the RNA from all the other biological junk in the sample. There are a bunch of different ways you can do this. When I was in college (~5 years ago), we did a [phenol-chloroform](https://en.wikipedia.org/wiki/Acid_guanidinium_thiocyanate-phenol-chloroform_extraction) extraction. What basically happens is that things other than RNA gets destroyed, and in the presence of certain chemicals, stuff gets separated out into kind of like a 3 layer bean dip. You extract your RNA from this, and you end up with a tiny pellet of nucleic acid.
* Because you are starting out with RNA, you actually have to do a *reverse transcription* PCR (RT-PCR), to first make complementary DNA (cDNA) from the RNA. Once you have your cDNA template, you can do regular PCR. You just give it some primers (short strands of DNA to give the reaction a starting point), some basic building blocks and an enzyme that assembles all this stuff together for you. Then you run this through a bunch of cycles at a certain temperature, which essentially amounts to Xeroxing your template over and over until you have a ton of copies.
* The key part here is that you choose the right primers (essentially fragments of the virus) that will bind to the cDNA of your sample. In other words, if there is no COVID-19 in the sample, then you won't get anything.

In [1]:
#hide

import requests
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

from IPython.display import HTML
%matplotlib inline

In [2]:
#hide

# get states data
states_url = 'https://covidtracking.com/api/states/daily'

r = requests.get(states_url)
states_df = pd.DataFrame(r.json())
states_df['date'] = pd.to_datetime(states_df.date, format='%Y%m%d')
states_df = states_df[(states_df.state == 'CA') | (states_df.state == 'NY')]

CA = states_df[states_df.state == 'CA']
NY = states_df[states_df.state == 'NY']

In [3]:
#hide

# specify colors for categories
domain = ['pending', 'positive', 'negative']
range_ = ['#f6a4ec', '#5433ff', '#97fbd1']

# convert wide-form into long-form
area = alt.Chart(CA).transform_fold(
    ['pending', 'positive', 'negative']
).mark_area(opacity=0.4, interpolate='basis').encode(
    alt.X('date:T', axis=alt.Axis(format=('%b %d'))),
    alt.Y('value:Q', stack=None),
    color=alt.Color('key:N', scale=alt.Scale(domain=domain, range=range_))
).properties(
    width=700,
    title='COVID-19 testing in California since Mar 4'
)

# add line on top of area chart
line = alt.Chart(CA).transform_fold(
    ['pending', 'positive', 'negative']
).mark_line(interpolate='basis', strokeWidth=2).encode(
    alt.X('date:T', axis=alt.Axis(format=('%b %d'))),
    alt.Y('value:Q', stack=None),
    color=alt.Color('key:N', scale=alt.Scale(domain=domain, range=range_))
)

# create selection that picks the nearest point from the date
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

# show the value of the cursor when you move it
selectors = alt.Chart(CA).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# draw points on the area chart
points = area.mark_point(fill='#ff4c4d', stroke='#ff4c4d').encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# create text labels by the points
text = area.mark_text(fill='black', align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# draw a line where the cursor is
rule = alt.Chart(CA).mark_rule(color='black').encode(
    x='date:T',
).transform_filter(
    nearest
)

# bind all elements together and customize top-level configurations
testChart = alt.layer(area, line, selectors, points, text, rule).configure_title(
    fontSize=25,
    color='#7f7f7f',
    anchor='start',
    dx=10
).configure_axis(
    grid=False
).configure_axisY(
    labelPadding=5,
    domainOpacity=0,
    orient='right',
    title=None
).configure_axisX(
    labelPadding=5,
    title=None
).configure_view(
    strokeWidth=0
).configure_legend(
    title=None,
    labelFontSize=15,
    labelColor='#7f7f7f',
    symbolType='square',
    orient='top',
    padding=10
)

> Tip: Hover over chart to interact

In [4]:
#hide_input

testChart

alt.LayerChart(...)

# Deaths and positive cases each day
> The step size represents quantity added each day

In [5]:
#hide

# create step graphs to compare increase in death/positive cases in CA and NY
domain = ['CA', 'NY']
range_ = ['#ff8f8f', '#036aff']

deaths = alt.Chart(states_df).transform_fold(
    ['CA', 'NY']
).mark_line(interpolate='step-after').encode(
    alt.X('date:T', axis=alt.Axis(format=('%b %d'))),
    alt.Y('deathIncrease:Q'),
    color=alt.Color('state:N', scale=alt.Scale(domain=domain, range=range_))
).properties(
    title='Deaths added each day',
    width=700
)

positive = alt.Chart(states_df).transform_fold(
    ['CA', 'NY']
).mark_line(interpolate='step-after').encode(
    alt.X('date:T', axis=alt.Axis(format=('%b %d'))),
    alt.Y('positiveIncrease:Q'),
    color=alt.Color('state:N', scale=alt.Scale(domain=domain, range=range_))
).properties(
    title='Positive cases added each day',
    width=700
)

# add text labels over steps
deathsText = deaths.mark_text(
    align='center',
    baseline='bottom',
    dx=8,
    dy=-5
).encode(
    text='deathIncrease'
)

positiveText = positive.mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    text='positiveIncrease'
)

deathsAdded = deaths + deathsText
positiveAdded = positive + positiveText

In [6]:
#hide

# create a function with custom top level chart configs
def makeChart(chart):
  
  return (
    chart.configure_title(
      fontSize=17,
      color='#7f7f7f',
      anchor='middle',
      dx=10
    ).configure_axis(
        grid=False
    ).configure_axisY(
        labels=False,
        domainOpacity=0,
        orient='right',
        ticks=False,
        title=None
    ).configure_axisX(
        labelPadding=5,
        title=None
    ).configure_view(
        strokeWidth=0
    ).configure_legend(
        title=None,
        labelFontSize=15,
        labelColor='#7f7f7f',
        symbolType='square',
        orient='left'
    )
  )

In [7]:
#hide_input

makeChart(deathsAdded)

alt.LayerChart(...)

In [8]:
#hide_input

makeChart(positiveAdded)

alt.LayerChart(...)

# Current hospital cases

In [9]:
#hide

# create bar chart for number of hospital cases
CAHospital_df = CA[CA.date >= '2020-03-26']
NYHospital_df = NY[NY.date >= '2020-03-26']


caHospital = alt.Chart(CAHospital_df).mark_bar(color='#ff8f8f').encode(
    alt.X('date:T', axis=alt.Axis(format='%b %d')),
    alt.Y('hospitalizedCurrently:Q'),
).properties(
    title='CA hospital cases'
)

nyHospital = alt.Chart(NYHospital_df).mark_bar(color='#036aff').encode(
    alt.X('date:T', axis=alt.Axis(format='%b %d')),
    alt.Y('hospitalizedCurrently:Q'),
).properties(
    title='NY hospital cases'
)

caText = alt.Chart(CAHospital_df).mark_text(color='black', dy=-10).encode(
    alt.X('date:T', axis=alt.Axis(format='%b %d')),
    alt.Y('hospitalizedCurrently:Q'),
    text=alt.Text('hospitalizedCurrently:Q')
)

nyText = alt.Chart(NYHospital_df).mark_text(color='black', dy=-10).encode(
    alt.X('date:T', axis=alt.Axis(format='%b %d')),
    alt.Y('hospitalizedCurrently:Q'),
    text=alt.Text('hospitalizedCurrently:Q')
)

# bind bar chart and text
californiaHospital = caHospital + caText
newyorkHospital = nyHospital + nyText

# horizontally concatenate charts
hospital = alt.hconcat(californiaHospital, newyorkHospital
).resolve_scale(
    y='shared'
)

In [10]:
#hide_input

makeChart(hospital)

alt.HConcatChart(...)

{{'Data is sourced from [The COVID Tracking Project](https://covidtracking.com/).' | fndetail: 1}}